In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
                    
import os
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score, precision_score, recall_score

In [4]:
from keras.layers import Input
from keras import Model
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D,GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import Callback

Using TensorFlow backend.


In [5]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()
from string import punctuation

import re
import os
import gc

import matplotlib.pyplot as plt

In [7]:
#print(os.listdir("../input"))

In [9]:
df = pd.read_csv('E:/Anaconda3/quora_train.csv')
df_test = pd.read_csv('E:/Anaconda3/quora_test.csv')

In [10]:
print("Columns :", df.columns)
print("Row 0 :")
print("qid :", df.iloc[0]['qid'])
print("question_text : ", df.iloc[0]['question_text'])
print("target :", df.iloc[0]['target'])

Columns : Index(['qid', 'question_text', 'target'], dtype='object')
Row 0 :
qid : 00002165364db923c7e6
question_text :  How did Quebec nationalists see their province as a nation in the 1960s?
target : 0


In [12]:
n_rows = len(df)
n_insincere = sum(df['target'])
print(n_rows)
print(n_insincere)

label_repart = pd.DataFrame(data={"" :[n_rows - n_insincere, n_insincere]}, index = [str(n_rows - n_insincere) + ' sincere questions', str(n_insincere) + ' insincere question'])
label_repart.plot(kind='pie', title='Labels repartition, ratio ' + str(round(n_insincere / n_rows,2)*100) + "%", subplots=True, figsize=(8,8))

1306122
80810


array([<matplotlib.axes._subplots.AxesSubplot object at 0x00000252DB674518>], dtype=object)

In [13]:
insincere_question = df[df['target'] == 1]['question_text'].values
for i in range(10):
    print(insincere_question[i])

Has the United States become the largest dictatorship in the world?
Which babies are more sweeter to their parents? Dark skin babies or light skin babies?
If blacks support school choice and mandatory sentencing for criminals why don't they vote Republican?
I am gay boy and I love my cousin (boy). He is sexy, but I dont know what to do. He is hot, and I want to see his di**. What should I do?
Which races have the smallest penis?
Why do females find penises ugly?
How do I marry an American woman for a Green Card? How much do they charge?
Why do Europeans say they're the superior race, when in fact it took them over 2,000 years until mid 19th century to surpass China's largest economy?
Did Julius Caesar bring a tyrannosaurus rex on his campaigns to frighten the Celts into submission?
In what manner has Republican backing of 'states rights' been hypocritical and what ways have they actually restricted the ability of states to make their own laws?


In [14]:
def clean_review(review_col):
    review_corpus=[]
    stops = set(stopwords.words("english"))
    for i in range(0,len(review_col)):
        review=str(review_col[i])
        review=re.sub('[^a-zA-Z]',' ',review)
        word_token = word_tokenize(str(review).lower())
        #review = [word for word in word_token if word not in stops]
        #review=' '.join(review)
        review=[lemma.lemmatize(w) for w in word_token if w not in stops]
        review=' '.join(review)
        review_corpus.append(review)
        #if i % 1000 == 0:
           #print(i/len(review_col)) 
    return review_corpus

In [16]:
df['clean_question']=clean_review(df['question_text'].values)

In [17]:
df_test['clean_question']=clean_review(df_test['question_text'].values)

In [18]:
#all_words=' '.join(df['clean_question'].values)
#all_words=word_tokenize(all_words)
#dist=FreqDist(all_words)
#num_unique_word=len(dist)
num_unique_word =166289

In [19]:
df.head()

,qid,question_text,target,clean_question
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0,quebec nationalist see province nation
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0,adopted dog would encourage people adopt shop
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0,velocity affect time velocity affect space geo...
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0,otto von guericke used magdeburg hemisphere
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0,convert montra helicon mountain bike changing ...


In [20]:
#r_len=[]
#for text in df['clean_question'].values:
#    word=word_tokenize(text)
#    l=len(word)
#    r_len.append(l)    
#MAX_QUESTION_LEN=np.max(r_len)
MAX_QUESTION_LEN=125

In [21]:
MAX_FEATURES = num_unique_word
MAX_WORDS = MAX_QUESTION_LEN

In [22]:
y_train = df['target'].values
X_train_text = df['clean_question'].values
X_test_text = df_test['clean_question'].values
print(X_train_text.shape,y_train.shape)
print(X_val_text.shape,y_val.shape)

(1306122,) (1306122,)


NameError: name 'X_val_text' is not defined

In [23]:
X_train_text, X_val_text, y_train, y_val = train_test_split(X_train_text, y_train, test_size=0.1)

In [24]:
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(list(X_train_text))
X_train = tokenizer.texts_to_sequences(X_train_text)
X_val = tokenizer.texts_to_sequences(X_val_text)
X_test = tokenizer.texts_to_sequences(X_test_text)

In [25]:
X_train = sequence.pad_sequences(X_train, maxlen=MAX_WORDS)
X_val = sequence.pad_sequences(X_val, maxlen=MAX_WORDS)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_WORDS)
print(X_train.shape,X_val.shape)

(1175509, 125) (130613, 125)


In [26]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')
    
def get_embed_mat(EMBEDDING_FILE, max_features,embed_dim):
    # word vectors
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE, encoding='utf8'))
    print('Found %s word vectors.' % len(embeddings_index))

    # embedding matrix
    word_index = tokenizer.word_index
    num_words = min(max_features, len(word_index) + 1)
    all_embs = np.stack(embeddings_index.values()) #for random init
    embedding_matrix = np.zeros((len(word_index) + 1, embed_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    max_features = embedding_matrix.shape[0]
    
    return embedding_matrix

In [ ]:
gloveEmbed = get_embed_mat('E:/Anaconda3/glove.840B.300d.txt', MAX_FEATURES, 300)